# Анализ данных выпускников университета

Цель работы: проанализировать данные о выпускниках университета, описать поведение выпускников на рынке труда и определить их особенности (в частности, для настройки таргета и для рекламы)

## План работы**:**
- Открыть файл с данными, изучить общую информацию, описать полученный датасет
- Провести предобработку данных (проверка на пропуски, дубли, типы данных в столбцах)
- Провести исследовательский анализ данных, построить графики:

  1) определить, после какого срока работы выпускники чаще меняют рабочее место (управленцам, которых тоже обучают в университете, нужно знать "опасные" периоды, когда вероятность потерять сотрудника выше);

  2) выяснить, есть ли связь между полом выпускника и выбранной им ведущей дисциплиной (есть ли смысл рекламировать какое-то направление больше среди женщин или мужчин);

  3) выявить, в каких городах находится больше выпускников (во-первых, найти конкретные города, на которые стоит настраивать таргет, во-вторых, на основе индекса развития города спрогнозировать, насколько его жители заинтересованы в высшем образовании);

  4) получить информацию о профессиональном успехе учеников (если она положительная, её можно использовать в рекламе университета);

  5) обучить модель для прогнозирования значения таргет.
- Проанализировать датасет и описать выводы;

### 1. Откройте файл с данными и изучите общую информацию. 

In [ ]:
import numpy as np #для анализа
import pandas as pd
import re
import plotly.express as px #для визуализации
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sklearn #для ml
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import neighbors

In [ ]:
%%bash
git clone https://github.com/vlad-kalambet/dbi.git  # Клонирование репозитория
mv ./dbi/* ./                                       # Перемещение всех файлов из папки dbi в рабочую директорию для удобного взаимодействия
rm -rf ./dbi                                        # Удаление пустой папки 

Cloning into 'dbi'...


In [ ]:
data = pd.read_csv('Данные о выпускниках университета.csv')

In [ ]:
data #рассматриваем head и tail таблицы

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,7386,city_173,0.878,Male,No relevent experience,no_enrollment,Graduate,Humanities,14,NaN,NaN,1,42,1.0
19154,31398,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,NaN,NaN,4,52,1.0
19155,24576,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,4,44,0.0
19156,5756,city_65,0.802,Male,Has relevent experience,no_enrollment,High School,NaN,<1,500-999,Pvt Ltd,2,97,0.0


In [ ]:
data.info() #общая информация: присутствуют значения nan, некоторые типы данных не соответствуют реальности (обозначены как object, должны быть int)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevent_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  target                  19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

###Подробные данные о столбцах

In [ ]:
data.columns[data.isna().any()] #столбцы, в которых есть хотя бы 1 значение nan

Index(['gender', 'enrolled_university', 'education_level', 'major_discipline',
       'experience', 'company_size', 'company_type', 'last_new_job'],
      dtype='object')

In [ ]:
columns_to_look_at = ['gender', 'relevent_experience', 'enrolled_university', 'education_level',
                      'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job', 'target']

for col in columns_to_look_at: #смотрим на уникальные значения в выбранных столбцах
  print(col, data[col].unique(), sep=': ', end='\n')

gender: ['Male' nan 'Female' 'Other']
relevent_experience: ['Has relevent experience' 'No relevent experience']
enrolled_university: ['no_enrollment' 'Full time course' nan 'Part time course']
education_level: ['Graduate' 'Masters' 'High School' nan 'Phd' 'Primary School']
major_discipline: ['STEM' 'Business Degree' nan 'Arts' 'Humanities' 'No Major' 'Other']
experience: ['>20' '15' '5' '<1' '11' '13' '7' '17' '2' '16' '1' '4' '10' '14' '18'
 '19' '12' '3' '6' '9' '8' '20' nan]
company_size: [nan '50-99' '<10' '10000+' '5000-9999' '1000-4999' '10/49' '100-500'
 '500-999']
company_type: [nan 'Pvt Ltd' 'Funded Startup' 'Early Stage Startup' 'Other'
 'Public Sector' 'NGO']
last_new_job: ['1' '>4' 'never' '4' '3' '2' nan]
target: [1. 0.]


In [ ]:
len(data['enrollee_id'].unique()) == len(data) #проверяем, все ли номера выпускников уникальные - да, дублей нет

True

In [ ]:
len(data['city'].unique()) #данные по скольким городам присутствуют в таблице - 123

123

In [ ]:
data.describe() #сводка по столбцам с числовыми значениями

,enrollee_id,city_development_index,training_hours,target
count,19158.000000,19158.000000,19158.000000,19158.000000
mean,16875.358179,0.828848,65.366896,0.249348
std,9616.292592,0.123362,60.058462,0.432647
min,1.000000,0.448000,1.000000,0.000000
25%,8554.250000,0.740000,23.000000,0.000000
50%,16982.500000,0.903000,47.000000,0.000000
75%,25169.750000,0.920000,88.000000,0.000000
max,33380.000000,0.949000,336.000000,1.000000


### Вывод

*   Присутствуют значения NaN в столбцах, записанных в список columns_with_nan: gender, enrolled_university, education_level, major_discipline, experience, company_size, company_type, last_new_job
*   Мы не будем удалять строки со значениями NaN по всей таблице, так как потеряем слишком много данных. Поскольку в большинстве случаев мы будем рассматривать отдельные избранные столбцы, мы будем избавляться от пустых значений в них, чтобы сохранить как можно больше информации
*   Столбцы experience, company_size, last_new_job, city содержат числовые значения, но записаны как object, требуют обработки на следующем шаге
*   Все остальные столбцы не содержат аномалий и готовы к работе

### Шаг 2. Предобработка данных

**Обработка столбца experience**. Переводу в int мешают знаки < (в сочетании '< 1') и > (в сочетании '> 20'). Поскольку мы не знаем точно опыт работы выпускников с такими показателями, но при анализе опыт работы должен быть отсортирован, заменяем <1 на условный 0, а >20 на 21. NaN пока оставляем

In [ ]:
data.experience[data['experience'] == '>20'] = 21 #замена >20 на 21
data.experience[data['experience'] == '<1'] = 0 #замена <1 на 0
data.experience = data.experience.apply(lambda x: int(x) if str(x).isdigit() else x) #преобразуем в числа всё, что возможно

<ipython-input-11-49ffc5bd3043>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.experience[data['experience'] == '>20'] = 21 #замена >20 на 21
<ipython-input-11-49ffc5bd3043>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.experience[data['experience'] == '<1'] = 0 #замена <1 на 0


In [ ]:
data['experience'].unique() #значения nan сохранились

array([21., 15.,  5.,  0., 11., 13.,  7., 17.,  2., 16.,  1.,  4., 10.,
       14., 18., 19., 12.,  3.,  6.,  9.,  8., 20., nan])

**Обработка столбца company_size**. Значения состоят из промежутков с количеством работников: <10, 10/49 (должно быть 10-49), 50-99, 100-500 (должно быть 100-499), 500-999, 1000-4999, 5000-9999, 10000+

Ранжировать эти значения и использовать в графиках в таком виде будет неудобно, поэтому мы обозначим каждую группу минимальным числом: 1 для группы <10, 10 для группы 10-49 и так далее

In [ ]:
data.company_size[data['company_size'] == '<10'] = 1 #меняем промежутки на цифры - нижнюю границу промежутка
data.company_size[data['company_size'] == '10/49'] = 10
data.company_size[data['company_size'] == '50-99'] = 50
data.company_size[data['company_size'] == '100-500'] = 100
data.company_size[data['company_size'] == '500-999'] = 500
data.company_size[data['company_size'] == '1000-4999'] = 1000
data.company_size[data['company_size'] == '5000-9999'] = 5000
data.company_size[data['company_size'] == '10000+'] = 10000
data.company_size = data.company_size.astype(pd.Int64Dtype(), errors='ignore') #переводим всё в int

<ipython-input-13-546c4f1b3ccd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.company_size[data['company_size'] == '<10'] = 1 #меняем промежутки на цифры - нижнюю границу промежутка
<ipython-input-13-546c4f1b3ccd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.company_size[data['company_size'] == '10/49'] = 10
<ipython-input-13-546c4f1b3ccd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.company_size[data['company

In [ ]:
data['company_size'].unique() #значения изменились, nan сохранились

<IntegerArray>
[<NA>, 50, 1, 10000, 5000, 1000, 10, 100, 500]
Length: 9, dtype: Int64

**Обработка столбца *last_new_job***. Чтобы использовать значения в исследовании понадобится их сортировать: значения 'never' и '>4' мешают это сделать.

Столбец *last_new_job* показывает сколько (скорее всего лет) выпускник работает в текущей компании. Следовательно, never можно заменить на 0 (если не было новой работы, то нет и лет, проведенных в текущей компании).

'>4' оставляем неизменным, так как знак > при сортировке в любом случае будет помещен на последнее место. Перевод в целочисленные значения не обязателен, так как числа не дискретные, они будут использоваться как категории

In [ ]:
data.last_new_job[data['last_new_job'] == 'never'] = '0' #меняем never на 0

<ipython-input-15-bd81fd2a8722>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.last_new_job[data['last_new_job'] == 'never'] = '0' #меняем never на 0


In [ ]:
data['last_new_job'].unique() #значения изменились, nan сохранились

array(['1', '>4', '0', '4', '3', '2', nan], dtype=object)

**Обработка столбца city**. Поскольку все города представлены числами, а не реальными названиями, мы можем заменить их на целочисленные значения, которые затем удобно будет сортировать

In [ ]:
pattern = r'\d+' #с помощью регулярных выражений выбираем номера городов и переводим их в int
data.city = data.city.apply(lambda x: int(re.search(pattern, x).group()))

In [ ]:
data.city.unique() #проверяем: значения изменились и стали int

array([103,  40,  21, 115, 162, 176, 160,  46,  61, 114,  13, 159, 102,
        67, 100,  16,  71, 104,  64, 101,  83, 105,  73,  75,  41,  11,
        93,  90,  36,  20,  57, 152,  19,  65,  74, 173, 136,  98,  97,
        50, 138,  82, 157,  89, 150,  70, 175,  94,  28,  59, 165, 145,
       142,  26,  12,  37,  43, 116,  23,  99, 149,  10,  45,  80, 128,
       158, 123,   7,  72, 106, 143,  78, 109,  24, 134,  48, 144,  91,
       146, 133, 126, 118,   9, 167,  27,  84,  54,  39,  79,  76,  77,
        81, 131,  44, 117, 155,  33, 141, 127,  62,  53,  25,   2,  69,
       120, 111,  30,   1, 140, 179,  55,  14,  42, 107,  18, 139, 180,
       166, 121, 129,   8,  31, 171])

In [ ]:
data #таблица после преобразований

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,21.0,<NA>,NaN,1,36,1.0
1,29725,40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15.0,50,Pvt Ltd,>4,47,0.0
2,11561,21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5.0,<NA>,NaN,0,83,0.0
3,33241,115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,0.0,<NA>,Pvt Ltd,0,52,1.0
4,666,162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,21.0,50,Funded Startup,4,8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,7386,173,0.878,Male,No relevent experience,no_enrollment,Graduate,Humanities,14.0,<NA>,NaN,1,42,1.0
19154,31398,103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,14.0,<NA>,NaN,4,52,1.0
19155,24576,103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,21.0,50,Pvt Ltd,4,44,0.0
19156,5756,65,0.802,Male,Has relevent experience,no_enrollment,High School,NaN,0.0,500,Pvt Ltd,2,97,0.0


In [ ]:
data.info() #измененный тип данных в столбцах city, experience, company_size

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             19158 non-null  int64  
 1   city                    19158 non-null  int64  
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevent_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  float64
 9   company_size            13220 non-null  Int64  
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  target                  19158 non-null  float64
dtypes: Int64(1), float64(3), int64(3), obj

### Вывод
Данные предобработаны:
- столбец *city* обращён в числовые значения
- в столбце *experience* значения *\<1* и *>20* переведены в условные *0* и *21*; при анализе стоит учитывать, что эти показатели не реальны, а взяты только для удобства обработки
- в столбце *company_size* интервалы приведены к нижней границе
- в столбце *last_new_job* значение *\<1* преобразовано в *0*; все значения хранятся в строковом виде и могут быть отсортированы в нём

### 3. Исследовательский анализ данных.

###Задача 1. Как часто выпускники меняют работу (на основе данных по последней работе)?

In [ ]:
fig = px.histogram(data_frame=data.last_new_job) #строим гистограмму на основе столбца last_new_job
fig.update_layout(title="Как часто выпускники меняют работу",
                  xaxis_title="годы на последней работе",
                  yaxis_title="количество выпускников")
fig.update_xaxes(type='category', categoryorder='category ascending') #названия столбцов рассматриваем как категории, чтобы сохранить название >4
fig.show(bbox_inches='tight')

Больше половины всех исследованных провели на последнем рабочем месте около года (8040 человек). Вторая по численности группа - те, кто в последний раз находил новую работу больше 4 лет назад (3290 человек). Самые малые группы людей провели на последней работе 3-4 года (1024 и 1029 человек соответственно). 2452 человека вообще никогда не устраивались на работу.

Такие результаты могут объясняться как возрастом отобранных выпускников (возможно, это молодые люди, которые просто в силу возраста успели проработать только 1 год), так и общей тенденцией к смене рабочих мест: возможно, люди зачастую переходят на другое место после первого года работы. Чтобы подтвердить или опровергнуть эту гипотезу, рассмотрим гистограмму опыта работы людей, проведших на последнем рабочем месте 1 год.

In [ ]:
fig = px.histogram(data_frame=data[data['last_new_job'] == '1'].experience) #гистограмма показывает опыт работы людей, сменивших работу год назад
fig.update_layout(title="Опыт работы людей, которые сменили работу год назад",
                  xaxis_title="опыт работы в годах",
                  yaxis_title="количество людей")
fig.update_xaxes(type='category', categoryorder='total descending') #отсортировано по убывающему числу людей с заданным опытом
fig.show(bbox_inches='tight')

**Вывод:**

Гистограмма показывает, что значительная доля сменивших работу - люди с опытом более 20 лет. За ними следуют группы с опытом работы 4, 5, 3, 6 лет. Выпускники с опытом 1 год находятся на 11 месте, а без опыта - на 14.

Такое распределение подтверждает нашу гипотезу: значительная часть описанных выпускников решает сменить место после 1 года работы.

###Задача 2. Как соотносятся пол и выбор ведущего предмета? Каково соотношение полов среди выпускников?

In [ ]:
subj_gender = pd.concat([data['gender'], data['major_discipline']], axis=1).dropna() #извлекаем нужные столбцы, соединяем их и убираем все строки, где пропущено хотя бы 1 значение
subj_gender = subj_gender.groupby(['gender', 'major_discipline'], as_index=False).size() #группируем по полу и дисциплине и подсчитываем сумму

In [ ]:
fig = make_subplots(rows=1, cols=3,
                    specs=[[{"type": "pie"}, {"type": "pie"}, {"type": "pie"}]],
                    subplot_titles=("Женщины", "Мужчины", "Другие")) #строим три круговых диаграммы рядом

fig.add_trace(go.Pie(labels = subj_gender[subj_gender['gender'] == 'Female'].major_discipline,
             values = subj_gender[subj_gender['gender'] == 'Female'].iloc[:, 2]),

             row=1, col=1) #диаграмма по женщинам

fig.add_trace(go.Pie(labels = subj_gender[subj_gender['gender'] == 'Male'].major_discipline,
             values = subj_gender[subj_gender['gender'] == 'Male'].iloc[:, 2]),

             row=1, col=2) #диаграмма по мужчинам

fig.add_trace(go.Pie(labels = subj_gender[subj_gender['gender'] == 'Other'].major_discipline,
             values = subj_gender[subj_gender['gender'] == 'Other'].iloc[:, 2]),

             row=1, col=3) #диаграмма по другим

fig.update_layout(height=550, width=1000, title_text="Как соотносятся пол и выбор ведущего предмета")
fig.show(bbox_inches='tight')

In [ ]:
genders = data.groupby(['gender'], as_index=False).size() #круговая диаграмма показывает соотношение полов

fig = px.pie(genders, names='gender', values=genders.iloc[:, 1], title='Соотношение полов выпускников')
fig.show(bbox_inches='tight')

**Вывод:**

Диаграмма показывает, что большая часть выпускников из выборки (вне зависимости от пола) выбирала STEM (естественные науки и технологии) в качестве ведущего предмета. Расхождения заметны только в нескольких процентах: мужчины выбирают эту дисциплину в 89.2% случаев, женщины - 80.1%, другой пол - 80.4%. Таким образом, мужчины выбирают STEM чаще других групп приблизительно на 9%.

Женщины предпочитают гуманитарные науки (Humanities) в 10.2% случаев, что больше, чем показатели другого пола (5.59%) и значительно больше, чем показатели мужчин (3.77%).

Прочие предметы непопулярны в качестве ведущей дисциплины у всех полов. Выделить среди них можно разве что искусства (Arts) и другие предметы (Other), которые другой пол выбирает чаще (4.9%), чем мужчины и женщины (1.41% - 3.11%).

Судя по полученным результатам, мы работаем с данными о выпускниках университета, который специализируется в технических и естественнонаучных предметах.

Это предположение привело нас к новому вопросу: каково соотношение полов в выборке. Диаграмма показала, что на 90.2% датасет состоит из информации о мужчинах, на 8.45% - о женщинах и на 1.3% - о представителях другого пола/полов. Это открытие приводит на к следующему выводу: в данный университет, который приоритизирует предметы STEM, в большинстве случаев поступают/подают заявление мужчины.

###Задача 3. Какие города самые "образованные"? Как уровень развития города соотносится с уровнем образования?

In [ ]:
city_educ = pd.concat([data['city'], data['city_development_index'], data['education_level']], axis=1).dropna() #извлекаем нужные столбцы, соединяем их и убираем все строки, где пропущено хотя бы 1 значение
city_educ = city_educ.groupby(['city', 'city_development_index', 'education_level'], as_index=False).size()

In [ ]:
fig = px.scatter(city_educ, x='city', y="city_development_index", #ось x - названия города, ось y - индекс развития города
                 size= city_educ.iloc[:, 3], color="education_level") #цвет бабла - уровень образования, размер бабла - количество людей

fig.update_layout(title="Уровень образования в разных городах",
                  xaxis_title="город",
                  yaxis_title="индекс развития города")

fig.show(bbox_inches='tight')

Для первичного анализа была выбрана пузырьковая диаграмма, так как она позволяет посмотреть на данные в нескольких измерениях: продемонстрировать одновременно индекс развития города, его название, уровень образования и количество людей с этим уровнем образования.

Однако наша диаграмма проиллюстрировала данные не лучшим образом: значительная часть информации не видна, поскольку города в таблице представлены не равномерно, поэтому мы видим только показатели по городам 16, 21, 103, 160. Тем не менее диаграмма не бесполезна - по ней мы делаем вывод о неравномерности репрезентации городов.

In [ ]:
fig = px.histogram(city_educ, x="city_development_index", color="education_level", histnorm='percent')

fig.update_layout(title="Уровень образования и индекс развития города",
                  xaxis_title="индекс развития города",
                  yaxis_title="процент выпускников")

fig.show(bbox_inches='tight') #ось x - индекс развития города, ось y - процент людей, цвет - уровень образования

**Вывод:**

Чтобы получить более наглядные результаты мы рассматриваем гистограмму, где горизонтальная ось показывает индекс развития города, а вертикальная - процент разных ступеней образования.

Здесь действительно видна взаимосвязь между уровнем развития города и уровнем образования его жителей: в городах с индексом развития больше 0.65 людей, получивших образование больше, в том числе образование высокого уровня (Masters, Phd).

Из описанного нельзя делать выводы о зависимости одного показателя от другого (может быть, что в развитом городе больше людей решают получить образование, а может быть, что люди с высоким уровнем образования каким-то образом повышают индекс развития города). Тем не менее мы можем зафиксировать взаимосвязь между двумя показателями.

### Задача 4. Насколько успешны выпускники?

Университет может использовать информацию о профессиональном успехе своих выпускников в рекламе.

Поскольку у нас нет данных о нынешней трудоустроенности/зарплате/рейтинге выпускников, судить об их успешности мы можем только по косвенным показателям. Какие столбцы стоит выбрать?

- Какие данные не используем:

Первое предположение, что можно использовать уровень образования и опыт работы, мы отвергаем. Уровень образования говорит скорее о решении вести академическую карьеру, чем о профессиональном успехе: топовые специалисты в своей сфере не обязательно имеют учёные степени. Опыт работы, во-первых, больше говорит о возрасте выпускника, чем о его заслугах, во-вторых, этот опыт может относится к работе в других профессиональных сферах (а значит, мало связан с полученным в университете образованием).

- Какие данные используем:

1) релевантный опыт: из таблицы это неочевидно, но мы предполагаем, что здесь рассматривается релевантность полученному образованию. В таком случае, этот столбец говорит о том, работал ли выпускник по специальности. (Даже если наше предположение неверно, релевантный опыт всё равно остаётся показателем, важным для карьерного успеха);

2) тип компании: измерять профессиональный успех размером компании или её типом было бы некорректно, поскольку высокоуровневыми и развитыми могут быть как общественные организации, так и коммерческие, государственные или частные. Поэтому мы обратим внимание только на два типа: стартап на раннем этапе (early-stage startup) и финансируемый стартап (funded startup). Второй будем считать более успешным, поскольку его создатели смогли привлечь инвесторов.

In [ ]:
#анализ был проведен с помощью datalens от яндекса. ссылка на полученный дашборд:
#https://datalens.yandex.ru/racz5nyi7beeh-uspeh-vypusknikov
#полученные результаты дублируем и здесь (на всякий случай)

In [ ]:
rel_exp = data.groupby(['relevent_experience'], as_index=False).size()
startups = data[(data.company_type == 'Funded Startup') | (data.company_type == 'Early Stage Startup')].groupby(['company_type'], as_index=False).size()

In [ ]:
fig = make_subplots(rows=1, cols=2,
                    specs=[[{"type": "pie"}, {"type": "pie"}]],
                    subplot_titles=("Наличие релевантного опыта", "Финансируемые и начинающие стартапы")) #строим две круговые диаграммы рядом

fig.add_trace(go.Pie(labels = rel_exp.relevent_experience,
             values = rel_exp.iloc[:, 1]),

             row=1, col=1) #диаграмма по наличию релевантного опыта

fig.add_trace(go.Pie(labels = startups.company_type,
             values = startups.iloc[:, 1]),

             row=1, col=2) #диаграмма по стартапам

fig.update_layout(height=550, width=1000, title_text="Профессиональный успех выпускников")
fig.show(bbox_inches='tight')

**Вывод:**

- 72% выпускников имеют релевантный опыт (как мы предполагаем, опыт, связанный с полученным образованием);

- более 62% выпускников, трудоустроенных в стартапах, работают в успешных предприятиях, получающих финансирование.

###Задача 5. Построить модель для предсказания столбца target

Поскольку для обучения модели все данные должны быть обращены в числовые, мы преобразуем их на этом этапе

In [ ]:
data_no_nan = data.dropna() #убираем все строки, где есть хотя бы одно пустое значение
data_no_nan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8955 entries, 1 to 19155
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             8955 non-null   int64  
 1   city                    8955 non-null   int64  
 2   city_development_index  8955 non-null   float64
 3   gender                  8955 non-null   object 
 4   relevent_experience     8955 non-null   object 
 5   enrolled_university     8955 non-null   object 
 6   education_level         8955 non-null   object 
 7   major_discipline        8955 non-null   object 
 8   experience              8955 non-null   float64
 9   company_size            8955 non-null   Int64  
 10  company_type            8955 non-null   object 
 11  last_new_job            8955 non-null   object 
 12  training_hours          8955 non-null   int64  
 13  target                  8955 non-null   float64
dtypes: Int64(1), float64(3), int64(3), obje

In [ ]:
data_train = data_no_nan.loc[:, :'training_hours'] #разделяем данные на черты и столбец с целевой переменной
data_target = data_no_nan['target']

In [ ]:
data_train.gender[data_train['gender'] == 'Male'] = 0 #преобразуем пол в числа: мужской - 0, женский - 1, другой - 2
data_train.gender[data_train['gender'] == 'Female'] = 1
data_train.gender[data_train['gender'] == 'Other'] = 2

data_train.relevent_experience[data_train['relevent_experience'] == 'No relevent experience'] = 0 #отсутствие релевантного опыта - 0, наличие - 1
data_train.relevent_experience[data_train['relevent_experience'] == 'Has relevent experience'] = 1

data_train.enrolled_university[data_train['enrolled_university'] == 'no_enrollment'] = 0 #нет зачиления - 0, курс на неполнвй день - 1, на полный - 2
data_train.enrolled_university[data_train['enrolled_university'] == 'Part time course'] = 1
data_train.enrolled_university[data_train['enrolled_university'] == 'Full time course'] = 2

data_train.education_level[data_train['education_level'] == 'Primary School'] = 1 #уровни образования переведены в цифры
data_train.education_level[data_train['education_level'] == 'High School'] = 2
data_train.education_level[data_train['education_level'] == 'Graduate'] = 3
data_train.education_level[data_train['education_level'] == 'Masters'] = 4
data_train.education_level[data_train['education_level'] == 'Phd'] = 5

data_train.major_discipline[data_train['major_discipline'] == 'STEM'] = 1 #названия предметов переведены в цифры
data_train.major_discipline[data_train['major_discipline'] == 'Humanities'] = 2
data_train.major_discipline[data_train['major_discipline'] == 'Arts'] = 3
data_train.major_discipline[data_train['major_discipline'] == 'Business Degree'] = 4
data_train.major_discipline[data_train['major_discipline'] == 'No Major'] = 5
data_train.major_discipline[data_train['major_discipline'] == 'Other'] = 6

data_train.company_type[data_train['company_type'] == 'Other'] = 0 #типы компаний переведены в цифры
data_train.company_type[data_train['company_type'] == 'Early Stage Startup'] = 1
data_train.company_type[data_train['company_type'] == 'Funded Startup'] = 2
data_train.company_type[data_train['company_type'] == 'Public Sector'] = 3
data_train.company_type[data_train['company_type'] == 'NGO'] = 4
data_train.company_type[data_train['company_type'] == 'Pvt Ltd'] = 5

data_train.last_new_job[data_train['last_new_job'] == '>4'] = 5 #если последний раз выпускник сменял работу больше 4 лет назад, он будет обозначен условной цифрой 5

<ipython-input-52-a53133132478>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-52-a53133132478>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-52-a53133132478>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-52-a53133132478>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pand

In [ ]:
data_train = sklearn.preprocessing.scale(data_train) #предобработка данных

In [ ]:
break_gener = sklearn.model_selection.KFold(shuffle=True, n_splits=5, random_state=42) #разбиваем обучающую выборку на 5 частей для кросс-валидации

In [ ]:
k = 0 #выясняем на скольких соседях будут показаны лучшие результаты
max_res = 0

for i in range(1, 51): #рассматриваем варианты от 1 соседа до 50
    classif = sklearn.neighbors.KNeighborsClassifier(n_neighbors=i)
    res = sklearn.model_selection.cross_val_score(classif, data_train, y=data_target, cv=break_gener, scoring='accuracy').mean()
    if res > max_res:
      k = i
      max_res = res

In [ ]:
print(k, max_res) #на 23 соседях модель обучается успешнее всего; самая высокая точность - 85%

23 0.8499162479061976


In [ ]:
classif = sklearn.neighbors.KNeighborsClassifier(n_neighbors=23).fit(data_train, data_target) #классификатор, который можно использовать для предсказания столбца target на новых данных

### Вывод
1. **Как часто выпускники меняют работу?**
- значительная часть рассмотренной группы людей сменила работу около года назад;
- предположительно, проанализированные выпускники склонны задерживаться на одном рабочем месте надолго (более 4 лет) или сменять неподходящее место после первого года работы (а не после 2 или 3).
2. **Как соотносятся пол и выбор ведущего предмета?**
- подавляющее большинство изученных выпускников (больше 80%) выбирают STEM в качестве ведущей дисциплины. Вероятно, представленные данные - это информация о тех, кто учился в университете, подавал в него документы или обучался в школе при университете (поскольку у некоторых уровень образования - средняя школа);
- некоторые гендерные различия всё же наблюдаются: женщины выбирают гуманитарные науки чаще (10.2% по сравнению с 3.77% у мужчин);
- прочие предметы (бизнес, искусства и др.) не популярны у все полов (меньше 5% выпускников выбирают каждую дисциплину);
- на 90.2% датасет состоит из информации о мужчинах, то есть намного большей популярностью описываемый университет пользуется у мужчин.
3. **Какие города самые "образованные"? Как уровень развития города соотносится с уровнем образования?**
- в датасете города представлены в разной мере: города с номерами 16, 21, 103, 260 встречаются в десятки раз чаще других. Поэтому выбирать самые "образованные" города невозможно, так как о многих из них мы имеем недостаточно данных;
- существует связь между индексом развития города и количеством людей, получивших образование: в городах с индексом выше 0.65 проживает в разы больше образованных людей;
- полученные данные однако ничего не говорят о причинно-следственных связях.
4. **Насколько профессионально успешны выпускники?**
- 72% выпускников имеют релевантный опыт;
- более 62% выпускников, занятых в стартапах, работают в финансируемых компаниях.
5. **Обучить модель для предсказания целевой переменной (столбец target)**
- модель обучена с помощью метода ближайших соседей и с использованием кросс-валидации;
- точность предсказаний на тестовой выборке - 85%.

### 4. Общий вывод

Датасет предоставляет информацию о выпускниках одного университета.

По круговым диаграммам мы увидели, что 90.2% поступивших или подавших заявление в университет - мужчины. При этом среди всех выпускников независимо от пола самый популярный ведущий предмет - STEM. Из этого очевидно, что в данном университете особым уважением пользуются технические/естественнонаучные дисциплины, и ради них студенты поступают.

Среди выпускников есть люди с разным опытом работы (в том числе больше 20 лет) и сотрудники компаний разных типов и размеров.

Судя по распределению выпускников по городам, университет базируется в каком-то месте (предположительно, в городе 103), а не является онлайн-университетом, но привлекает студентов из разных населенных пунктов (всего 123 города с разными индексами развития).



Также была обучена модель для прогнозирования значений в столбце target (target может означать успешное или неуспешное окончание университета, способность найти работу за определённый срок после выпуска, согласие участвовать в мероприятиях университета после получения диплома - без дополнительного объяснения нельзя точно определить).
****
**Поведение выпускников на рынке труда:**

Выпускники склонны после первого года менять место работы, если оно им не подходит по каким-то причинам, или оставаться более, чем на 4 года. Период, когда "опасность" потерять сотрудника выше всего - промежуток между 1 и 2 годом работы.
****
**Целевая аудитория для рекламы университета:**

*Безопасная стратегия:*

  1) продвигать университет среди групп, похожих на тех, кто уже выбирал это учреждение, - среди мужчин из городов 16, 21, 103, 160;

  2) использовать самое популярное направление как флагманский продукт - STEM.

*Рискованная стратегия (если университет хочет расширить аудиторию):*

  1) **рекламировать университет среди женщин.** Это оправданный шаг, так как женщины в среднем получают высшее образование даже чаще мужчин (по данным Statista, в 2020 году по всему миру 41% женщин и 36% мужчин имели высшее образование). То есть, среди выпускников мало женщин не потому, что они не заинтересованы в учебе в университете, а по другим причинам (возможно, потому что они реже выбирают STEM как ведущую дисциплину, которая так популярна в рассмотренном университете: согласно исследованию Elsevier только 25% исследований по физике было опубликовано учеными-женщинами, но это число растёт в 9 из 14 изученных стран). Университет мог бы использовать эту глобальную тенденцию и расширить свою аудиторию;

  2) **развивать другие направления** (гуманитарные науки, искусства, бизнес). Предыдущее предложение кажется более оправданным и надежным по сравнению с этим. Развитие других направлений потребует ресурсов, которые придётся отнять у уже налаженного STEM. Другие направления, конечно, привлекут новых студентов, но только в случае, если их развитие будет успешным.
****
  **Данные об успехе выпускников для рекламы:**
  
  1) среди исследованных выпускников 72% имеют релевантный опыт, а значит, их шансы получить работу после обучения в университете высоки;

  2) среди выпускников, занятых в стартапах, более 62% работают в успешных организациях, уже получающих финансирование.